This script takes the 10X output from Experiment 1 and Experiment 2 and converts the 10X output into count matrices.

# Import Packages

In [1]:
import pandas as pd     # Package to deal with data frames
import scipy.io         # Used to read in the matrix.mtx file using 'scipy.io.mmread'
import os               # Sets working directory
import csv              # Used to read the barcodes.tsv and features.tsv files using list comprehensions
import gzip             # Unzips .gz files from 10X output
import numpy as np      # Used for mathematical functions
import anndata          # Scanpy pipeline
import scanpy as sc     # Scanpy pipeline

# Set the Current Working Directory

In [ ]:
%config Completer.use_jedi = False

os.chdir('mydirectory')
os.getcwd()

In [3]:
# All functions are loaded from a file called 'Functions.py'
%run scripts/Functions.py

# Define Samples

In [4]:
samples = ['C_I_1', 'C_I_2', 'C_I_3', 'F_I_4', 'F_I_5', 'F_I_6',
           'C_P_7', 'C_P_8', 'F_P_10', 'F_P_11', 'F_P_12']

# Build Scanpy/AnnData objects

In [5]:
# Initialize empty list to fill in later
Adatas = [0] * len(samples)

# Loop over each sample
for i,sample in enumerate(samples):
    
    print('\nWorking on sample {}'.format(sample))
    
    # Get the path to the 10X data
    data_path = 'data/CellRanger/{}/filtered_feature_bc_matrix'.format(sample)
    
    # Load the data as an AnnData object
    adata = sc.read_10x_mtx(data_path, gex_only = False)
    
    # Save the count matrix as a Numpy array
    adata.X = adata.X.toarray()
    adata.X = adata.X.astype(int)
    
    # Save to list
    Adatas[i] = adata
    
    # Print the data object
    print(Adatas[i])
    
    print('Finished with sample {}!\n'.format(sample))


Working on sample C_I_1
AnnData object with n_obs × n_vars = 2125 × 32301
    var: 'gene_ids', 'feature_types'
Finished with sample C_I_1!


Working on sample C_I_2
AnnData object with n_obs × n_vars = 8111 × 32302
    var: 'gene_ids', 'feature_types'
Finished with sample C_I_2!


Working on sample C_I_3
AnnData object with n_obs × n_vars = 7528 × 32302
    var: 'gene_ids', 'feature_types'
Finished with sample C_I_3!


Working on sample F_I_4
AnnData object with n_obs × n_vars = 2990 × 32301
    var: 'gene_ids', 'feature_types'
Finished with sample F_I_4!


Working on sample F_I_5
AnnData object with n_obs × n_vars = 5018 × 32301
    var: 'gene_ids', 'feature_types'
Finished with sample F_I_5!


Working on sample F_I_6
AnnData object with n_obs × n_vars = 4302 × 32301
    var: 'gene_ids', 'feature_types'
Finished with sample F_I_6!


Working on sample C_P_7
AnnData object with n_obs × n_vars = 1435 × 32301
    var: 'gene_ids', 'feature_types'
Finished with sample C_P_7!


Working on s

# Add Metadata for each sample

In [6]:
# The following antibodies were used for CITE seq
CITE_all = ['OT-1', 'CD28', 'OX40', '4-1BB', 'DR3', 'NKG2D', 'PD-1', 'TIGIT', 'LAG-3', 'CTLA4', 'Tim-3',
             'VEGFR2', 'CD39', 'Hashtag1', 'Hashtag2', 'Hashtag3', 'hB2-M']

# Loop over each sample
for i,sample in enumerate(samples):
    
    # Add a column name for the sample name
    Adatas[i].obs['sample'] = sample
    
    # Add a column for the genotype
    if sample[0] == 'C':
        Adatas[i].obs['genotype'] = 'Control'
    if sample[0] == 'F':
        Adatas[i].obs['genotype'] = 'Formate'
        
    # Add column for antibody
    if sample.split('_')[1] == 'I':
        Adatas[i].obs['antibody'] = 'Rat IgG2a'
    if sample.split('_')[1] == 'P':
        Adatas[i].obs['antibody'] = 'anti-PD-1'
        
    # Deal with CITE-seq data
    CITE_cols = IntersectionOfGenes(CITE_all, Adatas[i])
    
    # Extract the CITE-seq columns as a pandas DataFrame
    CITE = pd.DataFrame(index = Adatas[i].obs_names,
                        columns = CITE_cols,
                        data = Adatas[i][:, CITE_cols].X.astype(int))

    # Add the CITE-seq data to the metadata
    Adatas[i].obs = pd.concat([Adatas[i].obs, CITE], axis = 1)

    # Remove the columns from the count matrices
    genes_to_keep = [gene for gene in Adatas[i].var_names if gene not in CITE_cols]
    Adatas[i] = Adatas[i][:, genes_to_keep]
        
    # Add total number of counts for each cell
    Adatas[i].obs['nCounts'] = np.sum(Adatas[i].X, axis = 1).astype(int)
    
    # Add number of unique genes that are expressed for each cell
    Adatas[i].obs['nGenes'] = np.count_nonzero(Adatas[i].X, axis = 1)

Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.
Trying to set attribute `.obs` of view, copying.


# Add TCR Metadata

In [7]:
# Loop over each sample
for i,sample in enumerate(samples):
    
    print(sample)
    
    # Load the TCR data from the CreateTCRData.py pipeline
    TCR_data = 'data/CellRanger/{}/{}_TCR.csv'.format(sample, sample)
    TCR_data = pd.read_csv(TCR_data, index_col = 0)
    
    # Add the TCR data to the metadata
    Adatas[i].obs = pd.concat([Adatas[i].obs, TCR_data], axis = 1)
    
    # Save TCR and AB_status as categorical data
    for col in ['TCR', 'AB_status']:
        Adatas[i].obs[col] = Adatas[i].obs[col].astype('category')
        
    # Save clone size as integer value
    Adatas[i].obs['CloneSize_pre_filter'] = Adatas[i].obs['CloneSize_pre_filter'].astype(int)

C_I_1
C_I_2
C_I_3
F_I_4
F_I_5
F_I_6
C_P_7
C_P_8
F_P_10
F_P_11
F_P_12


# Print out number of cells for GEX data

In [8]:
for i,sample in enumerate(samples):
    rows = Adatas[i].shape[0]
    col = Adatas[i].shape[1]
    
    print('For sample {}, there are {} cells and {} genes.'.format(sample, rows, col))

For sample C_I_1, there are 2125 cells and 32285 genes.
For sample C_I_2, there are 8111 cells and 32285 genes.
For sample C_I_3, there are 7528 cells and 32285 genes.
For sample F_I_4, there are 2990 cells and 32285 genes.
For sample F_I_5, there are 5018 cells and 32285 genes.
For sample F_I_6, there are 4302 cells and 32285 genes.
For sample C_P_7, there are 1435 cells and 32285 genes.
For sample C_P_8, there are 2681 cells and 32285 genes.
For sample F_P_10, there are 2412 cells and 32285 genes.
For sample F_P_11, there are 1443 cells and 32285 genes.
For sample F_P_12, there are 1605 cells and 32285 genes.


# Print out number of cells for TCR data

In [9]:
for i,sample in enumerate(samples):

    # Count number of cells that has a TCR sequenced
    rows = len(Adatas[i][ Adatas[i].obs['AB_status'] == 'AB_cell' ])
    
    print('For sample {}, there are {} cells with a sequenced TCR.'.format(sample, rows))

For sample C_I_1, there are 580 cells with a sequenced TCR.
For sample C_I_2, there are 5631 cells with a sequenced TCR.
For sample C_I_3, there are 3598 cells with a sequenced TCR.
For sample F_I_4, there are 1505 cells with a sequenced TCR.
For sample F_I_5, there are 3224 cells with a sequenced TCR.
For sample F_I_6, there are 2858 cells with a sequenced TCR.
For sample C_P_7, there are 729 cells with a sequenced TCR.
For sample C_P_8, there are 2084 cells with a sequenced TCR.
For sample F_P_10, there are 1025 cells with a sequenced TCR.
For sample F_P_11, there are 697 cells with a sequenced TCR.
For sample F_P_12, there are 1091 cells with a sequenced TCR.


# Change Cell Barcodes

In [10]:
# This step is done as a way to prepare for data integration later on
for i,sample in enumerate(samples):
    
    # Split the old cell barcodes by the '-' character
    # Take the nucleotide part of the barcodes (by using [0])
    # Then add _i for each sample
    s_number = Adatas[i].obs['sample'].unique()[0].split('_')[2]
    Adatas[i].obs.rename(index = lambda x: x.split('-')[0] + '_' + str(s_number), inplace = True)

# Concatenate into Single Adata Object

In [12]:
# Create a Python generator
generator = (Adatas[i] for i in range(1, len(Adatas)))

# Concatenate into a single object
Adata = Adatas[0].concatenate(generator, index_unique = None)

# Rename the 'batch' column to 'sample_num'
Adata.obs = Adata.obs.rename(columns = {'batch':'sample_num'})

# Renumber the sample_num column
Adata.obs['sample_num'] = [int(x.split('_')[2]) for x in Adata.obs['sample']]

# Reorder columns
# Order by GEX/sample/TCR data
cols = ['sample', 'sample_num', 'genotype', 'antibody', 'TCR', 'AB_status', 'nCounts', 'nGenes', 'CloneSize_pre_filter'  ]

Adata.obs = Adata.obs[cols]

# Replace NaN values
for col in ['TCR', 'AB_status']:
    Adata.obs[col] = Adata.obs[col].fillna('notcr')
    
for col in ['CloneSize_pre_filter']:
    Adata.obs[col] = Adata.obs[col].fillna(0).astype(int)

Adata.obs

,sample,sample_num,genotype,antibody,TCR,AB_status,OT-1,CD28,OX40,4-1BB,...,Tim-3,VEGFR2,CD39,Hashtag1,Hashtag2,Hashtag3,hB2-M,nCounts,nGenes,CloneSize_pre_filter
Barcode,,,,,,,,,,,,,,,,,,,,,
AAACCTGCAAAGTGCG_1,C_I_1,1,Control,Rat IgG2a,notcr,notcr,10,310,608,221,...,138,77,302,6143,27,35,0,8060,2563,0
AAACCTGCACGAAAGC_1,C_I_1,1,Control,Rat IgG2a,notcr,notcr,2,12,21,40,...,27,0,55,1289,19,18,0,663,196,0
AAACCTGGTCACACGC_1,C_I_1,1,Control,Rat IgG2a,notcr,notcr,1,59,54,26,...,159,6,276,4286,32,28,0,774,299,0
AAACCTGTCCCTCAGT_1,C_I_1,1,Control,Rat IgG2a,CAASGTSSGQKLVF|CAWSLGVANTGQLYF,AB_cell,0,22,50,14,...,44,2,54,8,19,1473,0,9853,2977,20
AAACCTGTCGACCAGC_1,C_I_1,1,Control,Rat IgG2a,|CASSPRTGGFSGNTLYF,beta_chain_only,1,11,8,5,...,5,2,9,8,30,1442,0,4004,1421,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TTTGCGCTCGCGTTTC_12,F_P_12,12,Formate,anti-PD-1,CALSDHPGYQNFYF|CASSGQGAREQYF,AB_cell,18,34,15,66,...,59,7,227,8,6167,14,0,18219,3829,30
TTTGGTTCATTGGTAC_12,F_P_12,12,Formate,anti-PD-1,CAAKDYSNNRLTL|CASSRGTGEDTQYF,AB_cell,0,69,14,133,...,151,1,204,3509,86,37,0,35219,5903,26
TTTGGTTGTCCAACTA_12,F_P_12,12,Formate,anti-PD-1,CAMREGGDNYAQGLTF|CASSLLGGAETLYF-CASSLSPGTGGTETLYF,AB_cell,0,19,73,61,...,108,0,468,2768,28,32,0,19696,4480,92


# Save File

In [13]:
Adata.write_h5ad('data/ScanpyObjects/MergedData_prefilter.h5ad')

... storing 'sample' as categorical
... storing 'genotype' as categorical
... storing 'antibody' as categorical
... storing 'TCR' as categorical
... storing 'feature_types' as categorical
